In [1]:
from ALLCools.clustering import *
from ALLCools.mcds import MCDS
from wmb import brain, aibs, mm10

import pandas as pd
import anndata
import seaborn as sns
import matplotlib.pyplot as plt
from ALLCools.plot import *

import scanpy as sc
from wmb import cemba
import glob
import numpy as np


In [2]:
group_name = 'Slice9-10_NN'

In [4]:
_slice , cell_class = group_name.split('_')
if cell_class == 'NN':
    mc_type = 'CGN'
if cell_class == 'Neu':
    mc_type = 'CHN'

In [5]:
mc_cells = pd.read_csv('mc_cells.txt', index_col=0, header=None).index
mc_cells.name = 'cell'
mc_cells.size

30004

In [6]:
merfish_cells = pd.read_csv('merfish_cells.txt', index_col=0, header=None).index
merfish_cells.name = 'cell'
merfish_cells.size

30194

## get merfish adata

In [7]:
var_dim = 'gene'
cef = pd.read_csv('mC.CEF.csv', header=None, index_col=0).squeeze()
cef = cef[cef].index
cef.name = var_dim

In [8]:
cef_gene_neme = [mm10.gene_id_to_name(gene_id) for gene_id in cef]
merfish_ds = MCDS.open('/cemba/MERFISH/MERFISH.TotalWatershedCell.BasicFilter.zarr', var_dim=var_dim)
merfish_ds = merfish_ds.sel(gene = cef_gene_neme, cell = merfish_cells)
merfish_ds

/opt/conda/lib/python3.7/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing with an out-of-order index is generating 76 times more chunks
  value = value[(slice(None),) * axis + (subkey,)]
/opt/conda/lib/python3.7/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing with an out-of-order index is generating 276 times more chunks
  value = value[(slice(None),) * axis + (subkey,)]


<xarray.MCDS>
Dimensions:      (cell: 30194, gene: 448)
Coordinates:
    blank_count  (cell) uint32 0 0 1 1 1 0 0 2 0 1 0 2 ... 0 2 1 0 0 0 0 0 0 1 3
  * cell         (cell) <U51 'C3-R1-113536214012792458791364771000738170785' ...
  * gene         (gene) <U13 'Galnt9' 'Kirrel3' 'Npas3' ... 'Igsf3' 'Rxfp1'
    gene_id      (gene) <U18 'ENSMUSG00000033316' ... 'ENSMUSG00000034009'
    n_counts     (cell) uint32 62 155 143 259 92 163 234 ... 53 31 43 47 283 427
    n_genes      (cell) uint32 48 104 99 129 70 93 120 ... 44 30 35 41 139 158
Data variables:
    count        (cell, gene) uint32 dask.array<chunksize=(2, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  gene

In [9]:
qry_adata = merfish_ds.get_count_adata(da_name='count', loading_chunk=50000)
qry_adata = qry_adata[:, cef_gene_neme].copy()
qry_adata.var.index = cef

Loading chunk 0-30194/30194


In [10]:
qry_adata.X.data = qry_adata.X.data/np.repeat(qry_adata.obs['n_counts'].values, qry_adata.X.getnnz(axis=1)
                                     ) * qry_adata.obs['n_counts'].median()
sc.pp.log1p(qry_adata)

In [11]:
qry_adata.write_h5ad('merfish_input.h5ad')

In [12]:
qry_adata

AnnData object with n_obs × n_vars = 30194 × 448
    obs: 'blank_count', 'n_counts', 'n_genes'
    var: 'id'
    uns: 'log1p'

## get mc adata

In [13]:
var_dim = 'geneslop2k-vm23'
cef = pd.read_csv('mC.CEF.csv', header=None, index_col=0).squeeze()
cef = cef[cef].index
cef.name = var_dim

In [14]:
chrom_to_remove = ['chrX', 'chrY', 'chrM', 'chrL']
downsample = 1000
std_cutoff = 0.01

In [15]:
mcds = MCDS.open(cemba.CEMBA_SNMC_MCDS_PATH, var_dim=var_dim, use_obs=mc_cells)

In [16]:
mcds.add_feature_selection_column(cef, f'{var_dim}_cef')
adata = mcds.get_adata(mc_type=mc_type,
                       select_hvf=f'{var_dim}_cef',
                       split_large_chunks=False)

In [17]:
std_filter = adata.X.std(axis=0) > std_cutoff
adata._inplace_subset_var(std_filter)

In [18]:
# log mC fraction and scale features
log_scale(adata, with_mean=True)

# reverse mC fraction so its positively corr with RNA
adata.X *= -1

In [19]:
adata.write_h5ad('mc_input.h5ad')

In [20]:
adata

AnnData object with n_obs × n_vars = 30004 × 448
    var: 'chrom', 'cov_mean', 'end', 'start', 'cef'
    uns: 'log'